In [28]:
import pandas as pd
from os import path
import json

In [29]:
tables = json.loads(open(path.join('data','libraries-html-table.json')).read())
keys = list(tables.keys())
# pd.read_html(tables[key])
df = pd.concat([pd.read_html(tables[key])[0].assign(lib=key) for key in keys])
df

,Variable,Count of LSOAs,Min decile,lib
0,Barriers to Housing and Services,5,1,Osidge Library
1,Income Deprivation Affecting Older People (IDA...,2,2,Osidge Library
2,Employment,1,2,Osidge Library
3,Income,1,2,Osidge Library
4,Income Deprivation Affecting Children Index (I...,1,2,Osidge Library
...,...,...,...,...
0,Barriers to Housing and Services,5,1,Finchley Church End Library
1,Living Environment,2,2,Finchley Church End Library
2,Income,1,2,Finchley Church End Library
3,Income Deprivation Affecting Children Index (I...,1,2,Finchley Church End Library


In [30]:
text="""The Income Deprivation Domain measures the proportion of the population experiencing deprivation relating to low income. The definition of low income used includes both those people that are out-of-work, and those that are in work but who have low earnings (and who satisfy the respective means tests).

The Employment Deprivation Domain measures the proportion of the working-age population in an area involuntarily excluded from the labour market. This includes people who would like to work but are unable to do so due to unemployment, sickness or disability, or caring responsibilities.

The Education, Skills and Training Deprivation Domain measures the lack of attainment and skills in the local population. The indicators fall into two sub-domains: one relating to children and young people and one relating to adult skills. 

The Health Deprivation and Disability Domain measures the risk of premature death and the impairment of quality of life through poor physical or mental health. The domain measures morbidity, disability and premature mortality but not aspects of behaviour or environment that may be predictive of future health deprivation.

The Crime Domain measures the risk of personal and material victimisation at local level. 

The Barriers to Housing and Services Domain measures the physical and financial accessibility of housing and local services. The indicators fall into two sub-domains: ‘geographical barriers’, which relate to the physical proximity of local services, and ‘wider barriers’ which includes issues relating to access to housing such as affordability.

The Living Environment Deprivation Domain measures the quality of the local environment. The indicators fall into two sub-domains. The ‘indoors’ living environment measures the quality of housing; while the ‘outdoors’ living environment contains measures of air quality and road traffic accidents."""

In [31]:
text_list=text.split('\n\n')
res = {t[len('The '):t.index(' Domain')].replace(' Deprivation',''):t for t in text_list}
res

{'Income': 'The Income Deprivation Domain measures the proportion of the population experiencing deprivation relating to low income. The definition of low income used includes both those people that are out-of-work, and those that are in work but who have low earnings (and who satisfy the respective means tests).',
 'Employment': 'The Employment Deprivation Domain measures the proportion of the working-age population in an area involuntarily excluded from the labour market. This includes people who would like to work but are unable to do so due to unemployment, sickness or disability, or caring responsibilities.',
 'Education, Skills and Training': 'The Education, Skills and Training Deprivation Domain measures the lack of attainment and skills in the local population. The indicators fall into two sub-domains: one relating to children and young people and one relating to adult skills. ',
 'Health and Disability': 'The Health Deprivation and Disability Domain measures the risk of premat

In [32]:
df.Variable.unique()

array(['Barriers to Housing and Services',
       'Income Deprivation Affecting Older People (IDAOPI)', 'Employment',
       'Income', 'Income Deprivation Affecting Children Index (IDACI)',
       'Index of Multiple Deprivation (IMD)', 'Living Environment',
       'Crime', 'Education, Skills and Training'], dtype=object)

In [33]:
res['Income Deprivation Affecting Children Index (IDACI)'] = 'The Income Deprivation Affecting Children Index (IDACI) measures the proportion of all children aged 0 to 15 living in income deprived families. It is a subset of the Income Deprivation Domain which measures the proportion of the population in an area experiencing deprivation relating to low income. The definition of low income used includes both those people that are out-of-work, and those that are in work but who have low earnings (and who satisfy the respective means tests).'
res['Income Deprivation Affecting Older People (IDAOPI)'] = 'The Income Deprivation Affecting Older People Index (IDAOPI) measures the proportion of all those aged 60 or over who experience income deprivation. It is a subset of the Income Deprivation Domain which measures the proportion of the population in an area experiencing deprivation relating to low income. The definition of low income used includes both those people that are out-of-work, and those that are in work but who have low earnings (and who satisfy the respective means tests).'
res['Index of Multiple Deprivation (IMD)']= 'The Index of Multiple Deprivation (IMD) is an overall relative measure of deprivation constructed by combining seven domains of deprivation according to their respective weights'

In [34]:
[x for x in df.Variable.unique() if x not in res.keys()]

[]

In [35]:
from bs4 import BeautifulSoup

In [36]:
def fix_html(lib):
    _vars = df.Variable.unique()
    cdf = df[df.lib==lib]
    html = cdf.drop('lib', axis=1).to_html(index=False)
    soup = BeautifulSoup(html, 'html.parser')
    tds = soup.find_all('td')
    for td in tds:
        txt = td.contents[0]
        if txt in list(_vars):
            tag = soup.new_tag('a')
            tag.string=txt
            tag['class'] = 'tooltipped hover'
            tag['data-position']="top"
            tag['data-tooltip']=res[txt]
            td.string.replace_with(tag)

    return soup.prettify()

# fix_html('Osidge Library')
output={lib:fix_html(lib) for lib in df.lib.unique()}

In [40]:
with open(path.join('data','libraries-html-table.json'),'w') as f:
    f.write(json.dumps(output).replace('dataframe','striped'))